In [64]:
from gurobipy import Model, GRB, quicksum

In [65]:
# Initialize the model
m = Model("charging_stations")

In [66]:
# Indices for companies and sites
companies = [1, 2]
sites = list(range(1, 31))

In [67]:
# Parameters
fixed_cost = 2500000  # Fixed cost for building a station
variable_cost = 150000  # Variable cost per charging unit

In [68]:
# Decision variables
z = m.addVars(companies, sites, vtype=GRB.BINARY, name="z")
x = m.addVars(companies, sites, vtype=GRB.INTEGER, name="x")

In [69]:
# Objective function
m.setObjective(
    quicksum(2500000 * z[i, n] + 150000 * x[i, n] for i in companies for n in sites),
    GRB.MINIMIZE
)

In [70]:
# Constraints
# Each site can have at most one company building a charging station
for n in sites:
    m.addConstr(quicksum(z[i, n] for i in companies) <= 1)

# Constraint a: At least 14 charging stations must be built along the corridor
m.addConstr(quicksum(z[i, n] for i in companies for n in sites) >= 14)

# Constraint b: No more than 18 charging units can be built at any of the charging stations
for i in companies:
    for n in sites:
        m.addConstr(x[i, n] <= 18 * z[i, n])

# Constraint c: If a charging station is built at site 2, one must also be built at site 30
for i in companies:
    m.addConstr(z[i, 2] <= z[i, 30])

# Constraint d: If company 1 builds a charging station at site 8, it cannot build one at site 21
m.addConstr(z[1, 8] + z[1, 21] <= 1)

# Constraint e: The number of charging stations built by company 2 at sites 3, 11, 17, 19, and 23 must equal 4
m.addConstr(quicksum(z[2, n] for n in [3, 11, 17, 19, 23]) == 4)

# Constraint f: The number of charging stations built by both companies at sites 12, 15, and 18 cannot be less than 1
m.addConstr(quicksum(z[i, n] for i in companies for n in [12, 15, 18]) >= 1)

# Constraint g: If company 1 builds a charging station at site 1, they must also build stations at sites 26, 27, and 28
m.addConstrs((z[1, 1] <= z[1, n] for n in [26, 27, 28]))

# Constraint h: The number of charging units built at station 14 must equal the number built at station 22
m.addConstr(quicksum(x[i, 14] for i in companies) == quicksum(x[i, 22] for i in companies))


<gurobi.Constr *Awaiting Model Update*>

In [71]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 120 columns and 267 nonzeros
Model fingerprint: 0xbd2cb119
Variable types: 0 continuous, 120 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+05, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 3.500000e+07
Presolve removed 100 rows and 120 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3.5e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.500000000000e+07, best bound 3.500000000000e+07, gap 0.0000%


In [72]:
# Print the solution
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
    for i in companies:
        for n in sites:
            if z[i, n].x > 0:
                print(f'Company {i} builds a station at site {n} with {x[i, n].x} units.')
else:
    print('No optimal solution found.')

Optimal cost: 35000000.0
Company 1 builds a station at site 4 with -0.0 units.
Company 1 builds a station at site 5 with -0.0 units.
Company 1 builds a station at site 6 with -0.0 units.
Company 1 builds a station at site 15 with -0.0 units.
Company 1 builds a station at site 16 with -0.0 units.
Company 1 builds a station at site 28 with -0.0 units.
Company 1 builds a station at site 29 with -0.0 units.
Company 2 builds a station at site 7 with -0.0 units.
Company 2 builds a station at site 8 with -0.0 units.
Company 2 builds a station at site 9 with -0.0 units.
Company 2 builds a station at site 11 with -0.0 units.
Company 2 builds a station at site 17 with -0.0 units.
Company 2 builds a station at site 19 with -0.0 units.
Company 2 builds a station at site 23 with -0.0 units.
